### Example 2 - AI Attachment Reviewer

This example will walk you through the following steps:

1. Downloading attachments from a Feature Service
2. Use a LLM to analyze attachments (images & PDFs) and provide a text summary for each
3. Write that summary back to the feature for review in an ArcGIS Manager Instant Application


#### Download Feature Service Attachments

This helpful section of code was pulled from Rami's excellent sample here: https://github.com/ralouta/ArcGIS_Code_Repo/blob/main/src/scripts/Feature%20Service%20Management/download_attachments_fs.ipynb.


In [4]:
import toml
import sys
import os
import shutil
from arcgis.gis import GIS
from langchain_openai import AzureChatOpenAI

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from chains.image_extractor import ImageExtractChain
from chains.pdf_extractor import PDFExtractChain
from models.image_information import ImageInformation
from models.pdf_information import PDFInformation

In [5]:
# Connect to Esri Federal AGOL
gis = GIS("https://esrifederal.maps.arcgis.com", client_id="VEFjlNUX3GINnELq")

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://esrifederal.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=VEFjlNUX3GINnELq&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=DT6gFcoqaIVSVD0yYClHMuqG1RpKM9&allow_verification=false


c:\Users\rami8629\AppData\Local\ESRI\conda\envs\genai-auto-py3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'esrifederal.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
# clone over a feature service and a webmap to your own content in a new folder
source_group = gis.groups.get("78a63c27e59941bbbff470a3f62e2d92")
source_group_items = source_group.content()

folder_name = "SE Led Training GeoGenAI 2024"
gis.content.create_folder(folder_name)
gis.content.clone_items(items=source_group_items, copy_data=True, folder=folder_name)

print(
    "done cloning. please check your Content for a new folder and then make sure you locate the Item Id of the new feature service that was created. You will use that for the next step."
)

C:\Users\rami8629\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: create_folder is deprecated as of 2.3.0 and has be removed in 3.0.0. Use `gis.content.folders.create` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Folder already exists.


c:\Users\rami8629\AppData\Local\ESRI\conda\envs\genai-auto-py3\Lib\site-packages\arcgis\gis\__init__.py:5770: DeprecationWarning: invalid escape sequence '\s'
  """
c:\Users\rami8629\AppData\Local\ESRI\conda\envs\genai-auto-py3\Lib\site-packages\arcgis\gis\__init__.py:9219: DeprecationWarning: invalid escape sequence '\{'
  new_path = f"{current_path}\{title}" if current_path else title
c:\Users\rami8629\AppData\Local\ESRI\conda\envs\genai-auto-py3\Lib\site-packages\arcgis\gis\__init__.py:13883: DeprecationWarning: invalid escape sequence '\A'
  """
c:\Users\rami8629\AppData\Local\ESRI\conda\envs\genai-auto-py3\Lib\site-packages\arcgis\gis\__init__.py:16736: DeprecationWarning: invalid escape sequence '\P'
  """
c:\Users\rami8629\AppData\Local\ESRI\conda\envs\genai-auto-py3\Lib\site-packages\arcgis\_impl\common\_clone.py:2944: DeprecationWarning: invalid escape sequence '\W'
  name = re.sub("\W+", "_", name)
c:\Users\rami8629\AppData\Local\ESRI\conda\envs\genai-auto-py3\Lib\site-packag

Exception: You do not have permissions to access this resource or perform this operation.
(Error Code: 403)

In [6]:
# Get the feature layer item id from the previous cloning step.
item_id = "ac062cbb932b478f96075b4ae326447c"

# Get the item
item = gis.content.get(item_id)

feature_layer = item.layers[0]

# Create a directory to save attachments
output_dir = "attachments"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [7]:
# Function to download attachments
def download_attachments(feature_layer, output_dir):
    # Query all features
    features = feature_layer.query(where="1=1", out_fields="*").features

    for feature in features:
        object_id = feature.attributes[feature_layer.properties.objectIdField]
        globalid = feature.attributes[feature_layer.properties.globalIdField]
        attachments = feature_layer.attachments.get_list(object_id)

        for attachment in attachments:
            attachment_id = attachment["id"]
            attachment_name = attachment["name"]
            final_attachment_path = os.path.join(
                output_dir, f"{globalid}__{attachment_name}"
            )

            # Check if the attachment already exists
            if not os.path.exists(final_attachment_path):
                temp_dir = os.path.join(output_dir, f"temp_{object_id}")

                # Create a temporary directory to download the attachment
                if not os.path.exists(temp_dir):
                    os.makedirs(temp_dir)

                # Download the attachment to the temporary directory
                feature_layer.attachments.download(
                    oid=object_id, attachment_id=attachment_id, save_path=temp_dir
                )

                # Move the attachment from the temporary directory to the output directory with the new name
                temp_attachment_path = os.path.join(temp_dir, attachment_name)
                shutil.move(temp_attachment_path, final_attachment_path)

                # Remove the temporary directory
                shutil.rmtree(temp_dir)

                print(
                    f"Downloaded {attachment_name} for feature {object_id} as {final_attachment_path}"
                )
            else:
                print(
                    f"Attachment {attachment_name} for feature {object_id} already exists as {final_attachment_path}"
                )


# Download attachments
download_attachments(feature_layer, output_dir)

Attachment Brazil_Manaus_drought.jpg for feature 1 already exists as attachments\d51d99fe-6d6c-4b87-8192-a96e5259748e__Brazil_Manaus_drought.jpg
Attachment Brazil_temperature_anomaly_december.png for feature 2 already exists as attachments\ae7f789d-84fd-4dd1-9fcb-c78fdf233763__Brazil_temperature_anomaly_december.png
Attachment France_carte_pastilles_10x.png for feature 3 already exists as attachments\d12a33d7-972f-4ed8-bdbd-59eca23e784b__France_carte_pastilles_10x.png
Attachment Argentina_Informe meteorologico 16-17 diciembre 2023.pdf for feature 4 already exists as attachments\6aa762a8-dee6-4c3c-bb29-82f847faef50__Argentina_Informe meteorologico 16-17 diciembre 2023.pdf
Attachment Slovenia_neurja_2jun2022.pdf for feature 5 already exists as attachments\f70a5812-b3a4-4a51-bf62-cec6ee70fe5d__Slovenia_neurja_2jun2022.pdf


### Use a LLM to analyze the attachments


In [8]:
azure_config = toml.load("config.toml")["configs"][0]
llm = AzureChatOpenAI(
    openai_api_version=azure_config["api_version"],
    azure_deployment=azure_config["deployment_name"],
    api_key=azure_config["api_key"],
    azure_endpoint=azure_config["api_endpoint"],
    model=azure_config["model_name"],
    model_name=azure_config["model_name"],
    temperature=0,
)
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-47dff8d4-b10c

In [ ]:
# loop through attachments folder
for root, dirs, files in os.walk(output_dir):
    for file in files:
        global_id = file.split("__")[0]
        print("=====================================")
        print(f"Processing attachment for global id: {global_id}")

        file_path = os.path.join(root, file)

        attachment_description = None
        if file.upper().endswith(".PNG") or file.upper().endswith(".JPG"):
            print("analyzing image attachment ...")
            iec_chain = ImageExtractChain(model=llm, image_path=file_path)
            result = iec_chain.extract_from_image()
            attachment_description = iec_chain.format_for_attachment(result)
        else:
            print("analyzing pdf attachment ...")
            pdf_chain = PDFExtractChain(model=llm, pdf_path=file_path)
            result = pdf_chain.extract_from_pdf()
            attachment_description = pdf_chain.format_for_attachment(result)

        if attachment_description is None:
            continue

        feature = [
            {
                "attributes": {
                    feature_layer.properties.globalIdField: global_id,
                    "AIReview": attachment_description,
                }
            }
        ]

        print("Updating AI Review...")
        feature_layer.edit_features(
            use_global_ids=True,
            updates=feature,
        )

        print(f"Updated AI Review for global id: {global_id}")
        print("=====================================")
        print("")

Processing attachment for global id: 6aa762a8-dee6-4c3c-bb29-82f847faef50
analyzing pdf attachment ...
Updating AI Review...
Updated AI Review for global id: 6aa762a8-dee6-4c3c-bb29-82f847faef50

Processing attachment for global id: ae7f789d-84fd-4dd1-9fcb-c78fdf233763
analyzing image attachment ...
Updating AI Review...
Updated AI Review for global id: ae7f789d-84fd-4dd1-9fcb-c78fdf233763

Processing attachment for global id: d12a33d7-972f-4ed8-bdbd-59eca23e784b
analyzing image attachment ...
Updating AI Review...
Updated AI Review for global id: d12a33d7-972f-4ed8-bdbd-59eca23e784b

Processing attachment for global id: d51d99fe-6d6c-4b87-8192-a96e5259748e
analyzing image attachment ...
Updating AI Review...
Updated AI Review for global id: d51d99fe-6d6c-4b87-8192-a96e5259748e

Processing attachment for global id: f70a5812-b3a4-4a51-bf62-cec6ee70fe5d
analyzing pdf attachment ...
Updating AI Review...
Updated AI Review for global id: f70a5812-b3a4-4a51-bf62-cec6ee70fe5d

